Python (как и большинство языков) не управляет файлами напрямую — вместо этого он взаимодействует с операционной системой (ОС), которая отвечает за реальные операции чтения/записи. Взаимодействие происходит через **дескриптор**.
1. Когда вы открываете файл (при помощи функции `open()`), Python запрашивает у ОС доступ к файлу.
2. ОС возвращает файловый дескриптор — это числовой идентификатор, который система использует для отслеживания открытых файлов.
3. В Python этот дескриптор оборачивается в файловый объект, то есть экземпляр отдельного класса (например, `_io.TextIOWrapper`), который даёт удобные методы для работы с файлами (`read()`, `write()`), которые мы разберем позже.

# Открытие файлов в Python
Прежде чем работать с файлом, его нужно открыть с помощью функции `open()`.

**Синтаксис**

    file = open(path, mode, encoding)  

Аргументы функции:
* `path` — путь к файлу (единственный обязательный аргумент);

* `mode` — режим доступа ('r', 'w', 'a', 'b' и др., по умолчанию 'r');

* `encoding` — кодировка (например, 'utf-8'): важно для корректной работы с кириллицей и другими не-[ASCII](https://en.wikipedia.org/wiki/ASCII) символами;

In [ ]:
# попробуем открыть файл
file = ('file_sample.txt')

# Пути к файлам

1. **Абсолютные и относительные пути**

 * Абсолютный путь — полный путь от корня файловой системы:

    Windows: C:\\Users\\user\\data.txt

    Linux/MacOS: /home/user/data.txt

 * Относительный путь — путь относительно текущей директории скрипта:

    Например, если скрипт лежит в ~/Code/, а файл — в ~/Code/files/data.txt, то относительный путь: files/data.txt.

2. **Особенности путей в Windows vs. Unix**

В Windows используется обратный слеш \, который в Python-строках интерпретируется как начало escape-последовательности (например, \n — перенос строки).

**Как избежать проблем:**

 * Экранировать слеши: "C:\\\\Users\\\\file.txt"

 * Использовать raw-строки: r"C:\Users\file.txt"

 * Использовать / (Python автоматически преобразует): "C:/Users/file.txt"

В Linux/MacOS используется прямой слеш /, и таких проблем нет.

# Как Python работает с файлами

В отличие от текстовых редакторов, Python не позволяет произвольно перемещаться по файлу — он читает данные последовательно, как поток. Важные особенности:

1. **Линейное чтение**

 * файл воспринимается как последовательность символов/строк
 *после чтения строки возврата к ней нет (если не переоткрыть файл заново)
 *после полного чтения файл нужно закрывать (или использовать seek(0))

2. **Основные режимы работы**

```
Режим | Действие       | Если файл существует       | Если файла нет
------|----------------|----------------------------|----------------
'r'   | Чтение         | Читает с начала            | Ошибка (FileNotFoundError)
'w'   | Запись         | Полностью перезаписывает   | Создает новый
'a'   | Дозапись       | Добавляет в конец          | Создает новый
```

3. **Важные нюансы**
* В режиме 'w':
 * Файл создается по указанному пути (включая нужное расширение)
 * Все существующие данные безвозвратно удаляются сразу при открытии
 * Папки в пути должны существовать заранее
* В режиме 'a':
 * Курсор всегда находится в конце файла
 * Идеально для логов и накопления данных
 * Гарантированно не повредит существующую информацию

In [ ]:
# чтение (только для существующих файлов)
with open('data.txt', 'r') as f:
    content = f.read()

# запись (пересоздает файл)
with open('output.txt', 'w') as f:
    f.write("Новые данные")

# дозапись (без риска потерять данные)
with open('log.txt', 'a') as f:
    f.write("Новая запись\n")

**ЗАПОМНИТЬ**: всегда используйте with — это гарантирует корректное закрытие файла, даже если при работе возникнет ошибка



# Аргумент encoding в Python

1. Общая информация

* Параметр encoding принимает строку с названием кодировки:

    open('file.txt', 'r', encoding='utf-8')

Критически важен для корректной работы с текстами на русском и других языках.

2. Особенности для Windows

 В Windows всегда явно указывайте encoding='utf-8'. Причины:

 * Системная кодировка по умолчанию (cp1251 или cp866) может исказить юникод-символы.
 * UTF-8 поддерживает все языки и спецсимволы (например, лингвистическую нотацию: ɑ̃, ç, ɪ).

3. Почему UTF-8?
* Python 3 использует UTF-8 как кодировку по умолчанию для исходного кода и строк.
* Это единственная кодировка, которая:
 * Корректно работает с кириллицей, иероглифами, диакритическими знаками.
 * Совместима со всеми ОС (Linux/MacOS и так используют её по умолчанию).
 * Не требует BOM (Byte Order Mark) в отличие от UTF-16/32.

**Важно для вас как для лингвистов:**

* Для корпусов и словарей всегда используйте UTF-8.
* Если файл сохранён в другой кодировке (например, cp1251), указывайте её явно

**Важно**: Даже если ваш скрипт работает без encoding на Mac/Linux, явное указание utf-8 сделает код переносимым между ОС.

# Чтение и запись в файл

Итак, открываем несуществующий файл:

In [ ]:
file = open('test.txt', 'w', encoding='utf8')

## Чтение

**Как читать**:

1. `file.read()`
* Возвращает: всё содержимое как одну строку (str)
* Особенности:
 * Сохраняет \n (переносы строк)
 * Не подходит для больших файлов (загружает всё в память)
2. `file.readlines()`
* Возвращает: список строк (list[str])
* Особенности:
 * Каждый элемент списка (кроме последнего) оканчивается на \n
 * Тоже загружает весь файл в память
3. `file.readline()`
* Возвращает: одну строку (str)
* Особенности:
 * При повторном вызове читает следующую строку
 * Сохраняет \n в конце

4. Итерация по файлу через генератор: `[line for line in file]`
* Рекомендуется для больших файлов (читает по одной строке)
* Особенности:
 * Не загружает весь файл в память
 * line содержит \n в конце

Лучше всего использовать способ **4** для объемных файлов. В противном случае вы, например, загружаете гигабайтовый текстовый файл в оперативную память махом, и ноутбук превращается в тыкву...

## Запись

Методы записи:
1. `file.write(text)`
* Записывает: строку (str) или байты (bytes)
* Особенности:
 * Не добавляет \n автоматически
 * Возвращает количество записанных символов

2. `file.writelines(lines)`
* Записывает: Список строк (list[str])
* Особенности:
 * Не добавляет \n между элементами списка
 * Элементы списка должны содержать \n явно

3. `print(..., file=f)`
* Плюсы:
 * Автоматически добавляет \n (как обычный print)
 * Поддерживает f-строки и форматирование

Запишем что-нибудь в наш открытый файл:

In [ ]:
file.write('Hello world! ')
print('\nА это принт напринтил', file=file)
file.writelines([word + '\n' for word in input().split()])  # я вручную прилепила \n каждому слову

 это список слов где каждое слово это отдельная строка


Чтобы записанная информация сохранилась в файл, необходимо его закрыть. Вообще файлы нужно за собой закрывать! Даже если вы только из них читаете, не забудьте выполнить команду:

    file.close()
    
Иначе ваш файл останется болтаться открытым в оперативе. Это как фантик за собой в мусорку не выкинуть.

In [ ]:
file.close()

Теперь можно считать все эти строчки обратно из файла:

In [ ]:
file = open('test.txt', 'r', encoding='utf8')
for line in file:
    print(line.rstrip())  # rstrip нужен, чтобы откусить \n

Hello world!
А это принт напринтил
это
список
слов
где
каждое
слово
это
отдельная
строка


In [ ]:
file.close()  # выкинем за собой фантик

## Работа с файловой системой с помощью модуля os

В Python встроены базовые операции для работы с файлами — их можно открывать, читать, записывать и закрывать прямо "из коробки". Однако для более сложных задач, таких как создание папок, удаление файлов и директорий или манипуляции с путями, уже потребуются дополнительные модули.

Самый простой и универсальный вариант — модуль os, который позволяет выполнять множество файловых операций: создавать и удалять папки (os.mkdir(), os.rmdir()), проверять существование файлов (os.path.exists()), работать с путями (os.path.join()) и даже удалять файлы (os.remove()).

Если же хочется более современного и удобного подхода, можно обратиться к модулю pathlib, появившемуся в Python 3.4. Он предлагает объектно-ориентированный интерфейс для работы с путями и файловой системой, делая код чище и читаемее. Например, вместо громоздких конкатенаций путей через os.path.join() можно использовать простой оператор / для объединения частей пути.

Таким образом, если нужно просто прочитать или записать данные — хватит стандартного open(), но для управления файловой структурой лучше держать под рукой os или pathlib.

## Команды модуля os

**os**

* os.listdir(path)

Возвращает список файлов и папок в указанной директории.

* os.remove(path)

Удаляет файл. Не работает для папок.

* os.rmdir(path)

Удаляет пустую папку. Если папка не пуста — будет ошибка.

* os.mkdir(path)

Создаёт папку, но только если все родительские директории существуют.

* os.makedirs(path)

Создаёт папку и все недостающие родительские папки в пути.


**os.path**

* `os.path.exists(path)`

Проверяет существование файла или папки. Возвращает True/False.

* `os.path.isdir(path)`

Проверяет, является ли путь папкой.

* `os.path.isfile(path)`

Проверяет, является ли путь файлом.

* `os.path.isabs(path)`

Проверяет, является ли путь абсолютным (начинается с C:\ в Windows или /home/ в Linux/Mac).

* `os.path.abspath(path)`

Преобразует относительный путь в абсолютный.

* `os.path.splitext(path)`

Разделяет путь на имя файла (без расширения) и расширение.

* `os.path.split(path)`

Разделяет путь на директорию и имя файла.

* `os.path.join(path1, path2, ...)`

Объединяет части пути с учётом правил ОС.

Ограничение: Не работает с буквой диска (C:) напрямую.

Все функции чувствительны к регистру в Unix-системах и нечувствительны в Windows.



### Форматы файлов, сериализация

Мы с вами поговорили о разных форматах файлов, которые используются при работе со скриптами. Файлы, которые умеет обрабатывать питон, находятся на своего рода шкале по человекочитаемости:
1. .txt файлы - легко читаются человеком, трудно читаются скриптами (потому что не структурированы)
2. .json, .csv файлы - могут читаться как человеком, так и машиной
3. бинарные файлы - не предназначены для чтения человеком

Работать с обычными текстовыми файлами мы с вами уже умеем и научимся это делать еще виртуознее, когда освоим регулярки.

#### JSON

json - Java Script Object Notation; первоначально создавался для другого ЯП, но может быть использован и для типов питона. Это такой формат, в котором объекты питона записываются в машиночитаемом виде, но при этом могут читаться и человеком. Запись данных в машиночитаемом виде называется сериализацией: когда мы считываем такие файлы снова программой, не нужно их специально парсить (явно для человека).

С json-файлами работает модуль json, который является стандартным модулем из библиотек питона.

In [ ]:
import json

Нам для работы достаточно знать четыре функции:

- `json.load(file)`
- `json.dump(object, file, ensure_ascii=False, indent=4)`
- `json.loads(string)`
- `json.dumps(object)`

load загружает файл, dump сохраняет объект в файл, loads десереализует строку в объект питона, а dumps, наоборот, сериализует.

In [ ]:
with open('new.json') as file:  # я не указываю кодировку, потому что пользуюсь unix системой, но в Windows не забывайте об этом
    data = json.load(file)

In [ ]:
"""Такой способ чтения иногда бывает нужен, если в json не один объект, а много строк с объектами"""
data = []

with open('google.json') as file:
    for line in file:
        data.append(json.loads(line))

In [ ]:
with open('new2.json', 'w', encoding='utf8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

Когда сохраняем файл в формате json, последние два параметра необязательны, но лучше всегда указывать ensure_ascii=False (чтобы записывать в utf8); indent - это отступы внутри файла, если не указать этот параметр, весь объект запишется в одну строчку. 4 - это количество пробелов в отступе.

#### Бинарные файлы, pickle, dill

Быстрее и лучше всего машина читает бинарные файлы, записывать которые умеет стандартный модуль `pickle`. Такие файлы не читаются человеком (практически). Также можно использовать библиотеку dill (ее надо установить: `pip install dill`), у обоих модулей примерно одинаковый синтаксис.

In [ ]:
import pickle

pickle.dump(data, open('data', 'wb'))  # wb - режим записи бинарника. Не забывайте про b
data = pickle.load(open('data', 'rb'))  # rb - режим чтения бинарника. Есть также ab

In [ ]:
import dill

dill.dump(data, open('data', 'wb'))
data = dill.load(open('data', 'rb'))